In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait


In [4]:
#service = Service(ChromeDriverManager().install())
#driver = webdriver.Chrome()
#options = webdriver.ChromeOptions()
#options.binary_location = "/dados/produtos/advocacia/chrome-headless-shell/linux-130.0.6723.58/chrome-headless-shell-linux64/chrome-headless-shell"
#options.binary_location = "/Users/vagner/chrome-headless-shell/mac-130.0.6723.69/chrome-headless-shell-mac-x64/chrome-headless-shell"
#options.add_argument("--headless=new")
#options.enable_downloads = True
#options.browser_version = 'stable'
#options.add_experimental_option('excludeSwitches', ['disable-popup-blocking'])
#options.proxy = Proxy({ 'proxyType': ProxyType.MANUAL, 'httpProxy' : 'http.proxy:1234'})
#driver = webdriver.Chrome(options=options)


In [32]:
def getDriver():
    service = Service(ChromeDriverManager().install())
    options = webdriver.ChromeOptions()
    options.enable_downloads = True
    driver = webdriver.Chrome(service=service, options=options)
    driver.set_window_size(1024, 768)
    return driver



def getDriverHeadless():
    service = Service(ChromeDriverManager().install())
    options = webdriver.ChromeOptions()
    options.binary_location = "/Users/vagner/chrome-headless-shell/mac-130.0.6723.69/chrome-headless-shell-mac-x64/chrome-headless-shell"
    options.enable_downloads = True
    options.add_argument("--headless=new")
    driver = webdriver.Chrome(service=service, options=options)
    driver.set_window_size(1024, 768)
    return driver



def searchProcesso(processo):
    driver = getDriverHeadless()
    # encontra um processo
    driver.get("https://esaj.tjsp.jus.br/cpopg/open.do")
    tipoProcesso = driver.find_element(By.ID, "radioNumeroAntigo").click()
    numProcesso = driver.find_element(By.ID, "nuProcessoAntigoFormatado").send_keys(processo)
    btnSubmit = driver.find_element(By.ID, "botaoConsultarProcessos").click()
    # coleta as informações sobre o processo
    partes_envolvidas = driver.find_element(By.ID, "tablePartesPrincipais").text
    foro = driver.find_element(By.ID, "foroProcesso").text
    vara = driver.find_element(By.ID, "varaProcesso").text
    juiz = driver.find_element(By.ID, "juizProcesso").text
    classe = driver.find_element(By.ID, "classeProcesso").text
    assunto = driver.find_element(By.ID, "assuntoProcesso").text
    driver.execute_script("document.getElementById('tabelaTodasMovimentacoes').style.display='block';")
    movimentacoes_container = driver.find_element(By.ID, "tabelaTodasMovimentacoes")
    movim_individual = movimentacoes_container.find_elements(By.CLASS_NAME, "containerMovimentacao")
    movimentacoes = []
    for mov in movim_individual:
        movimentacoes.append(mov.text)
    driver.quit()
    # formatacao resposta
    resultados = {
        "partes_envolvidos": partes_envolvidas, 
        "foro": foro, 
        "vara": vara, 
        "juiz": juiz, 
        "classe": classe, 
        "assunto": assunto, 
        "historico": movimentacoes
    }
    return resultados





In [34]:


processo_numero = "1005493-05.2019.8.26.0024"
processo = searchProcesso(processo_numero)

processo


{'partes_envolvidos': 'Reqte  Unimil Indústria e Comércio de Peças e Máquinas Agrícolas Ltda\nAdvogado:  Guilherme Plaça Pinto  \nReqdo  Gildo J. Vianna Transportes Eirelli - Me\nAdvogado:  Fernando França Teixeira de Freitas  ',
 'foro': 'Foro de Andradina',
 'vara': '1ª Vara',
 'juiz': 'Paulo Victor Alvares Gonçalves',
 'classe': 'Monitória',
 'assunto': 'Compra e Venda',
 'historico': ['25/10/2023 Arquivado Definitivamente',
  '25/10/2023 Certidão de Cartório Expedida\nCertidão de Cartório - CUSTAS - Certidão de Pagamento de Custas e Arquivamento - Cível - 61615',
  '27/09/2023 Execução/Cumprimento de Sentença Iniciada (o)\n0002752-67.2023.8.26.0024 - Cumprimento de sentença',
  '27/09/2023 Execução/Cumprimento de Sentença Iniciada (o)\n0002750-97.2023.8.26.0024 - Cumprimento de sentença',
  '22/09/2023 Certidão de Publicação Expedida\nRelação: 0710/2023 Data da Publicação: 26/09/2023 Número do Diário: 3827',
  '22/09/2023 Remetido ao DJE\nRelação: 0710/2023 Teor do ato: Vistos. Ciê

In [52]:

def printResult(processo):
    print(f"Classe: {processo['classe']}\n")
    print(f"Assunto: {processo['assunto']}\n")
    print(f"Foro: {processo['foro']}\n")
    print(f"Vara: {processo['vara']}\n")
    print(f"Juiz: {processo['juiz']}\n")
    print(f"Histórico de movimentações:")
    for h in processo['historico']:
        print()
        print(h)


def formatResult(processo):
    result = []
    result.append(f"Classe: {processo['classe']}\n")
    result.append(f"Assunto: {processo['assunto']}\n")
    result.append(f"Foro: {processo['foro']}\n")
    result.append(f"Vara: {processo['vara']}\n")
    result.append(f"Juiz: {processo['juiz']}\n")
    result.append(f"Histórico de movimentações:")
    for h in processo['historico']:
        result.append("\n")
        result.append(h)
    return "".join(result)


formatResult(processo)


'Classe: Monitória\nAssunto: Compra e Venda\nForo: Foro de Andradina\nVara: 1ª Vara\nJuiz: Paulo Victor Alvares Gonçalves\nHistórico de movimentações:\n25/10/2023 Arquivado Definitivamente\n25/10/2023 Certidão de Cartório Expedida\nCertidão de Cartório - CUSTAS - Certidão de Pagamento de Custas e Arquivamento - Cível - 61615\n27/09/2023 Execução/Cumprimento de Sentença Iniciada (o)\n0002752-67.2023.8.26.0024 - Cumprimento de sentença\n27/09/2023 Execução/Cumprimento de Sentença Iniciada (o)\n0002750-97.2023.8.26.0024 - Cumprimento de sentença\n22/09/2023 Certidão de Publicação Expedida\nRelação: 0710/2023 Data da Publicação: 26/09/2023 Número do Diário: 3827\n22/09/2023 Remetido ao DJE\nRelação: 0710/2023 Teor do ato: Vistos. Ciência às partes do retorno dos autos da Superior Instância a esta Vara. No caso de cumprimento de sentença, observe o exequente os artigos 513 e seguintes. Em especial, os artigos 523 e 524, do Novo Código de Processo Civil. Deverá o credor solicitar o cumprimen